In [114]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import re


In [2]:
# Get Statistic Area level 2 information from csv file
sa2_df = pd.read_csv("resources/SA2_2016_AUST.csv")

# Filter the information to show Greater Perth Only
sa2_perth_df = sa2_df[sa2_df["GCCSA_NAME_2016"] == "Greater Perth"]

sa2_perth_df.head()


,SA2_MAINCODE_2016,SA2_5DIGITCODE_2016,SA2_NAME_2016,SA3_CODE_2016,SA3_NAME_2016,SA4_CODE_2016,SA4_NAME_2016,GCCSA_CODE_2016,GCCSA_NAME_2016,STATE_CODE_2016,STATE_NAME_2016,AREA_ALBERS_SQKM
1768,502011021,51021,Dawesville - Bouvard,50201,Mandurah,502,Mandurah,5GPER,Greater Perth,5,Western Australia,97.9248
1769,502011022,51022,Falcon - Wannanup,50201,Mandurah,502,Mandurah,5GPER,Greater Perth,5,Western Australia,9.5893
1770,502011023,51023,Greenfields,50201,Mandurah,502,Mandurah,5GPER,Greater Perth,5,Western Australia,9.9609
1771,502011024,51024,Halls Head - Erskine,50201,Mandurah,502,Mandurah,5GPER,Greater Perth,5,Western Australia,14.2469
1772,502011025,51025,Mandurah,50201,Mandurah,502,Mandurah,5GPER,Greater Perth,5,Western Australia,8.3954


In [9]:
income_dict = {"SA2 Name":[],"$1-$149 ($1-$7,799)":[],"$150-$299 ($7,800-$15,599)":[],"$300-$399 ($15,600-$20,799)":[],"$400-$499 ($20,800-$25,999)":[],
             "$500-$649 ($26,000-$33,799)":[], "$650-$799 ($33,800-$41,599)":[],"$800-$999 ($41,600-$51,999)":[],
             "$1,000-$1,249 ($52,000-$64,999)":[],"$1,250-$1,499 ($65,000-$77,999)":[],"$1,500-$1,749 ($78,000-$90,999)":[],
             "$1,750-$1,999 ($91,000-$103,999)":[],"$2,000-$2,499 ($104,000-$129,999)":[],"$2,500-$2,999 ($130,000-$155,999)":[],
             "$3,000-$3,499 ($156,000-$181,999)":[],"$3,500-$3,999 ($182,000-$207,999)":[],"$4,000-$4,499 ($208,000-$233,999)":[],
              "$4,500-$4,999 ($234,000-$259,999)":[],"$5,000-$5,999 ($260,000-$311,999)":[],
             "$6,000-$7,999 ($312,000-$415,999)":[],"$8,000 or more ($416,000 or more)":[]}

#   Data:  One family household with only family members present
for code in sa2_perth_df["SA2_MAINCODE_2016"]:
    sa2_code = str(code)
    report_range = np.arange(3,23)
    income_dict["SA2 Name"].append(sa2_perth_df.loc[sa2_perth_df["SA2_MAINCODE_2016"]==code,"SA2_NAME_2016"].values[0])
                                   
    for number in report_range:
        report_type = "{0:0=2d}".format(number)
        abs_url = f"http://stat.data.abs.gov.au/sdmx-json/data/ABS_C16_T21_SA/110.{report_type}.5.SA2.{sa2_code}/all?detail=Full&dimensionAtObservation=AllDimensions"
        response = requests.get(abs_url)
        data = response.json()
        income_dict[f'{data["structure"]["dimensions"]["observation"][1]["values"][0]["name"]}'].append(data["dataSets"][0]["observations"]["0:0:0:0:0:0"][0])




In [132]:
income_df = pd.DataFrame(income_dict)

In [133]:
income_df.columns = ["SA2 Name", "1-149(75)","150-299(225)","300-399(350)","400-499(450)","500-649(575)","650-799(725)",
                    "800-999(900)","1,000-1,249(1,125)","1,250-1,499(1,375)","1,500-1,749(1,625)","1,750-1,999(1,875)","2,000-2,499(2,250)",
                     "2,500-2,999(2,750)","3,000-3,499(3,250)","3,500-3,999(3,750)","4,000-4,499(4,250)",
                     "4,500-4,999(4,750)","5,000-5,999 (5,500)","6,000-7,999(7,000)","8,000 or more(8,000)"]

In [134]:
income_df["Total"] = income_df.iloc[:,1:21].sum(axis=1)

In [135]:
clean_income_df = income_df[income_df["Total"] > 100]

In [136]:
clean_income_df["Median"] =""

clean_income_df = clean_income_df.reset_index()


C:\Users\khiro\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [137]:
#Calculate Median from histgram value (search the index which exceeds (total number /2) )
for index_k,row in clean_income_df.iterrows():
    counter = 0
    i = 2
    while counter < row[22]/2:
        counter = counter + row[i]
        i = i + 1
    match = re.search(r"(\()(.*?)\)",row.index[i])
    clean_income_df.iloc[index_k,23] = int(match.groups(0)[1].replace(',',''))
    

In [138]:
clean_income_df

,index,SA2 Name,1-149(75),150-299(225),300-399(350),400-499(450),500-649(575),650-799(725),800-999(900),"1,000-1,249(1,125)",...,"2,500-2,999(2,750)","3,000-3,499(3,250)","3,500-3,999(3,750)","4,000-4,499(4,250)","4,500-4,999(4,750)","5,000-5,999 (5,500)","6,000-7,999(7,000)","8,000 or more(8,000)",Total,Median
0,0,Dawesville - Bouvard,17.0,12.0,16.0,54.0,64.0,218.0,117.0,167.0,...,160.0,98.0,69.0,43.0,35.0,28.0,12.0,0.0,1624.0,1875
1,1,Falcon - Wannanup,11.0,29.0,26.0,104.0,83.0,238.0,157.0,201.0,...,149.0,93.0,104.0,57.0,49.0,34.0,22.0,0.0,2023.0,1625
2,2,Greenfields,14.0,31.0,31.0,114.0,100.0,391.0,196.0,230.0,...,125.0,59.0,46.0,34.0,28.0,22.0,9.0,3.0,2108.0,1375
3,3,Halls Head - Erskine,32.0,42.0,53.0,139.0,144.0,482.0,313.0,408.0,...,357.0,221.0,212.0,149.0,130.0,73.0,53.0,8.0,4177.0,1875
4,4,Mandurah,20.0,24.0,36.0,105.0,116.0,251.0,178.0,224.0,...,101.0,59.0,55.0,29.0,26.0,18.0,16.0,0.0,1798.0,1375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,167,Rockingham,25.0,48.0,34.0,158.0,111.0,383.0,250.0,304.0,...,221.0,150.0,119.0,51.0,64.0,37.0,31.0,3.0,3047.0,1625
148,169,Safety Bay - Shoalwater,15.0,30.0,17.0,104.0,120.0,265.0,180.0,235.0,...,230.0,156.0,101.0,66.0,76.0,46.0,28.0,5.0,2543.0,1875
149,170,Singleton - Golden Bay - Secret Harbour,18.0,28.0,27.0,89.0,109.0,179.0,180.0,263.0,...,502.0,358.0,276.0,189.0,161.0,115.0,54.0,7.0,4082.0,2750
150,171,Waikiki,13.0,24.0,27.0,85.0,105.0,225.0,189.0,220.0,...,272.0,147.0,125.0,80.0,49.0,39.0,21.0,0.0,2631.0,2250
